# Projet - Mini-GPT : Génération de Texte

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Prérequis** : TPs 1-4 (Fondamentaux NLP, Attention, Multi-Head, Transformer)

---

## Objectifs du projet

Dans ce projet, vous allez :

**Partie 1 - From Scratch (1h)**
1. Comprendre la différence Encodeur vs Décodeur
2. Implémenter le masque causal
3. Construire un décodeur Transformer
4. Générer des noms fantasy de manière autoregressive

**Partie 2 - Fine-tuning GPT-2 (1h)**
5. Utiliser un modèle pré-entraîné (GPT-2)
6. Comparer les approches
7. Explorer les paramètres de génération

---

# PARTIE 1 : Décodeur From Scratch

---

## 0. Installation et imports

In [ ]:
!pip install torch transformers matplotlib numpy tqdm -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm.auto import tqdm

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

torch.manual_seed(42)

In [ ]:
# Télécharger le dataset de noms fantasy/celtes
!wget -q https://raw.githubusercontent.com/chris-lmd/transformers-but-sd/main/data/fantasy_names.txt
print("Dataset téléchargé !")

---

## 1. Théorie : Encodeur vs Décodeur

### BERT vs GPT

| Modèle | Architecture | Attention | Tâche |
|--------|-------------|-----------|-------|
| **BERT** | Encodeur | Bidirectionnelle | Classification, NER, QA |
| **GPT** | Décodeur | Causale (gauche→droite) | Génération de texte |

### Pourquoi un masque causal ?

Lors de l'entraînement de GPT, on veut prédire le prochain token. Mais l'attention standard permet à chaque position de "voir" toutes les autres positions, y compris les futures !

**Le masque causal** empêche chaque position de regarder les positions futures. C'est comme cacher la réponse pendant un examen.

```
Sans masque (BERT) :     Avec masque (GPT) :
Je  [voit tout]          Je  [voit: Je]
suis [voit tout]         suis [voit: Je, suis]
un  [voit tout]          un   [voit: Je, suis, un]
chat [voit tout]         chat [voit: Je, suis, un, chat]
```

---

## 2. Le Masque Causal

### Visualisation sans masque

In [ ]:
# Simulons une matrice d'attention sans masque
seq_len = 5
attn_scores = torch.randn(seq_len, seq_len)
attn_weights = F.softmax(attn_scores, dim=-1)

plt.figure(figsize=(6, 5))
plt.imshow(attn_weights.numpy(), cmap='Blues')
plt.colorbar()
plt.xlabel('Clé (K)')
plt.ylabel('Requête (Q)')
plt.title('Attention SANS masque (Encodeur)')
for i in range(seq_len):
    for j in range(seq_len):
        plt.text(j, i, f'{attn_weights[i, j]:.2f}', ha='center', va='center', fontsize=8)
plt.show()

print("Chaque position peut voir TOUTES les autres positions.")

### Exercice 1 : Créer le masque causal

Le masque causal est une matrice triangulaire supérieure remplie de `-inf` :

```
[[0, -inf, -inf, -inf],
 [0,  0,   -inf, -inf],
 [0,  0,    0,   -inf],
 [0,  0,    0,    0  ]]
```

Après softmax, les `-inf` deviennent 0, empêchant l'attention sur les positions futures.

In [ ]:
def create_causal_mask(seq_len):
    """
    Crée un masque causal de taille (seq_len, seq_len).
    Les positions futures (triangulaire supérieure) sont masquées avec -inf.
    
    Args:
        seq_len: Longueur de la séquence
    
    Returns:
        Tensor de shape (seq_len, seq_len) avec 0 pour les positions autorisées
        et -inf pour les positions masquées
    """
    # TODO: Créer le masque causal
    # Indice: Utilisez torch.triu() pour créer une matrice triangulaire supérieure
    # et torch.ones() pour créer la matrice de base
    
    mask = None  # À COMPLÉTER
    
    return mask

In [ ]:
# Test du masque
mask = create_causal_mask(5)
print("Masque causal:")
print(mask)

In [ ]:
# Visualisation avec masque
attn_scores_masked = attn_scores + mask
attn_weights_masked = F.softmax(attn_scores_masked, dim=-1)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Sans masque
axes[0].imshow(attn_weights.numpy(), cmap='Blues')
axes[0].set_title('SANS masque (Encodeur/BERT)')
axes[0].set_xlabel('Clé (K)')
axes[0].set_ylabel('Requête (Q)')

# Avec masque
axes[1].imshow(attn_weights_masked.numpy(), cmap='Blues')
axes[1].set_title('AVEC masque causal (Décodeur/GPT)')
axes[1].set_xlabel('Clé (K)')
axes[1].set_ylabel('Requête (Q)')

plt.tight_layout()
plt.show()

print("\nRemarquez que la partie supérieure droite est maintenant à 0 !")

**Question** : Pourquoi utilise-t-on `-inf` plutôt que `0` pour masquer ?

*Réponse* : ...

---

## 3. Dataset : Noms Fantasy

Nous allons entraîner notre Mini-GPT à générer des noms fantasy et celtes.

In [ ]:
# Charger les noms
with open('fantasy_names.txt', 'r', encoding='utf-8') as f:
    names = [line.strip() for line in f if line.strip() and not line.startswith('#')]

print(f"Nombre de noms: {len(names)}")
print(f"\nExemples:")
for name in names[::500][:10]:  # Un nom tous les 500
    print(f"  {name}")

In [ ]:
# Créer le vocabulaire (caractères uniques)
# On ajoute des tokens spéciaux
PAD_TOKEN = '<PAD>'
SOS_TOKEN = '<SOS>'  # Start of sequence
EOS_TOKEN = '<EOS>'  # End of sequence

# Collecter tous les caractères uniques
chars = set()
for name in names:
    chars.update(name)

chars = sorted(list(chars))
vocab = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN] + chars

# Créer les mappings
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = {i: ch for ch, i in char2idx.items()}

vocab_size = len(vocab)
print(f"Taille du vocabulaire: {vocab_size}")
print(f"Caractères: {vocab[:10]}...")

In [ ]:
# Statistiques sur les noms
lengths = [len(name) for name in names]
print(f"Longueur min: {min(lengths)}")
print(f"Longueur max: {max(lengths)}")
print(f"Longueur moyenne: {np.mean(lengths):.1f}")

plt.hist(lengths, bins=30, edgecolor='black')
plt.xlabel('Longueur du nom')
plt.ylabel('Fréquence')
plt.title('Distribution des longueurs de noms')
plt.show()

In [ ]:
# Dataset PyTorch pour la génération char-level
class NameDataset(Dataset):
    def __init__(self, names, char2idx, max_len=25):
        self.names = names
        self.char2idx = char2idx
        self.max_len = max_len
        self.pad_idx = char2idx[PAD_TOKEN]
        self.sos_idx = char2idx[SOS_TOKEN]
        self.eos_idx = char2idx[EOS_TOKEN]
    
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        name = self.names[idx]
        
        # Encoder: SOS + nom + EOS
        encoded = [self.sos_idx] + [self.char2idx[c] for c in name] + [self.eos_idx]
        
        # Padding
        if len(encoded) < self.max_len + 2:  # +2 pour SOS et EOS
            encoded += [self.pad_idx] * (self.max_len + 2 - len(encoded))
        else:
            encoded = encoded[:self.max_len + 2]
        
        # Input: tout sauf le dernier token
        # Target: tout sauf le premier token (décalé de 1)
        input_ids = torch.tensor(encoded[:-1], dtype=torch.long)
        target_ids = torch.tensor(encoded[1:], dtype=torch.long)
        
        return input_ids, target_ids

In [ ]:
# Créer le dataset
MAX_LEN = 25
BATCH_SIZE = 64

dataset = NameDataset(names, char2idx, MAX_LEN)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Vérification
input_ids, target_ids = dataset[0]
print(f"Input shape: {input_ids.shape}")
print(f"Target shape: {target_ids.shape}")

# Décoder pour vérifier
input_decoded = ''.join([idx2char[i.item()] for i in input_ids if i.item() != char2idx[PAD_TOKEN]])
target_decoded = ''.join([idx2char[i.item()] for i in target_ids if i.item() != char2idx[PAD_TOKEN]])
print(f"\nInput: {input_decoded}")
print(f"Target: {target_decoded}")

---

## 4. Le Décodeur Transformer

### Composants réutilisés des TPs

In [ ]:
# === Composants du Transformer (TPs 2-4) ===

class MultiHeadAttention(nn.Module):
    """Multi-Head Attention avec support du masque causal."""
    
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x, mask=None):
        B, S, _ = x.shape
        
        Q = self.W_q(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.d_k)
        
        # Appliquer le masque causal si fourni
        if mask is not None:
            scores = scores + mask
        
        attn = F.softmax(scores, dim=-1)
        out = (attn @ V).transpose(1, 2).contiguous().view(B, S, self.embed_dim)
        
        return self.W_o(out)


class FeedForward(nn.Module):
    """Feed-Forward Network."""
    
    def __init__(self, embed_dim, ff_dim=None, dropout=0.1):
        super().__init__()
        ff_dim = ff_dim or 4 * embed_dim
        self.net = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, embed_dim),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)


class PositionalEncoding(nn.Module):
    """Positional Encoding sinusoïdal."""
    
    def __init__(self, embed_dim, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1)])


print("Composants chargés !")

### Exercice 2 : Le Décodeur Transformer

Complétez la classe `TransformerDecoder` qui utilise :
- Une couche d'embedding
- Le positional encoding
- Plusieurs blocs de décodeur (avec masque causal)
- Une tête de prédiction (Linear → vocab_size)

In [ ]:
class DecoderBlock(nn.Module):
    """Un bloc du décodeur Transformer."""
    
    def __init__(self, embed_dim, num_heads, ff_dim=None, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.ff = FeedForward(embed_dim, ff_dim, dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Pre-norm architecture
        attn_out = self.attn(self.norm1(x), mask)
        x = x + self.dropout(attn_out)
        x = x + self.dropout(self.ff(self.norm2(x)))
        return x


class TransformerDecoder(nn.Module):
    """
    Mini-GPT : Décodeur Transformer pour la génération de texte.
    """
    
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, 
                 max_len=100, ff_dim=None, dropout=0.1, pad_idx=0):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.pad_idx = pad_idx
        
        # TODO: Créer les couches
        # 1. Embedding de taille (vocab_size, embed_dim)
        # 2. Positional Encoding
        # 3. Liste de DecoderBlock (num_layers blocs)
        # 4. LayerNorm finale
        # 5. Tête de prédiction (Linear: embed_dim → vocab_size)
        
        self.embedding = None  # À COMPLÉTER
        self.pos_encoding = None  # À COMPLÉTER
        self.layers = None  # À COMPLÉTER
        self.norm = None  # À COMPLÉTER
        self.head = None  # À COMPLÉTER
    
    def forward(self, x):
        B, S = x.shape
        
        # TODO: Implémenter le forward pass
        # 1. Embedding + scaling par sqrt(embed_dim)
        # 2. Positional encoding
        # 3. Créer le masque causal
        # 4. Passer par tous les DecoderBlock
        # 5. LayerNorm
        # 6. Tête de prédiction
        
        logits = None  # À COMPLÉTER
        
        return logits

In [ ]:
# Créer le modèle
EMBED_DIM = 64
NUM_HEADS = 4
NUM_LAYERS = 3
FF_DIM = 256

model = TransformerDecoder(
    vocab_size=vocab_size,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS,
    max_len=MAX_LEN + 2,
    ff_dim=FF_DIM,
    dropout=0.1,
    pad_idx=char2idx[PAD_TOKEN]
).to(device)

# Compter les paramètres
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Paramètres: {num_params:,}")

In [ ]:
# Test du modèle
test_input = torch.randint(0, vocab_size, (2, 10)).to(device)
test_output = model(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print(f"Expected: (2, 10, {vocab_size})")

---

## 5. Entraînement

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for input_ids, target_ids in tqdm(dataloader, desc="Training", leave=False):
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)
        
        optimizer.zero_grad()
        
        # Forward
        logits = model(input_ids)
        
        # Loss (reshape pour CrossEntropy)
        loss = criterion(
            logits.view(-1, logits.size(-1)),
            target_ids.view(-1)
        )
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [ ]:
# Configuration
EPOCHS = 50
LR = 1e-3

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss(ignore_index=char2idx[PAD_TOKEN])

# Historique
history = []

print("Début de l'entraînement...\n")

for epoch in range(EPOCHS):
    loss = train_epoch(model, dataloader, optimizer, criterion, device)
    history.append(loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss:.4f}")

print("\nEntraînement terminé !")

In [ ]:
# Visualisation
plt.figure(figsize=(10, 4))
plt.plot(history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Courbe d\'entraînement')
plt.grid(True)
plt.show()

---

## 6. Génération Autoregressive

### Exercice 3 : Implémenter la fonction de génération

La génération autoregressive fonctionne ainsi :
1. Commencer avec le token `<SOS>`
2. Prédire le prochain token
3. Ajouter ce token à la séquence
4. Répéter jusqu'à `<EOS>` ou longueur max

In [ ]:
@torch.no_grad()
def generate(model, prompt="", max_len=25, temperature=1.0):
    """
    Génère un nom de manière autoregressive.
    
    Args:
        model: Le modèle TransformerDecoder
        prompt: Début du nom (optionnel)
        max_len: Longueur maximale
        temperature: Contrôle la "créativité" (1.0 = normal)
    
    Returns:
        Le nom généré (string)
    """
    model.eval()
    
    # Initialiser avec SOS + prompt
    tokens = [char2idx[SOS_TOKEN]]
    for c in prompt:
        if c in char2idx:
            tokens.append(char2idx[c])
    
    # TODO: Génération autoregressive
    # Boucle jusqu'à max_len ou EOS:
    #   1. Convertir tokens en tensor
    #   2. Passer dans le modèle
    #   3. Prendre les logits du dernier token
    #   4. Appliquer temperature: logits = logits / temperature
    #   5. Softmax pour obtenir les probabilités
    #   6. Échantillonner le prochain token (torch.multinomial)
    #   7. Si EOS, arrêter
    #   8. Sinon, ajouter à tokens
    
    generated = ""  # À COMPLÉTER
    
    return generated

In [ ]:
# Tester la génération
print("=" * 50)
print("GÉNÉRATION DE NOMS FANTASY")
print("=" * 50)

# Sans prompt
print("\nSans prompt (température=1.0):")
for _ in range(10):
    name = generate(model, prompt="", temperature=1.0)
    print(f"  {name}")

# Avec prompt
print("\nAvec prompt 'Gwen':")
for _ in range(5):
    name = generate(model, prompt="Gwen", temperature=1.0)
    print(f"  {name}")

print("\nAvec prompt 'Mael':")
for _ in range(5):
    name = generate(model, prompt="Mael", temperature=1.0)
    print(f"  {name}")

---

## 7. Température et Créativité

### Exercice 4 : Explorer l'effet de la température

La température contrôle la "créativité" de la génération :
- **T < 1** : Plus conservateur, plus prévisible
- **T = 1** : Distribution normale
- **T > 1** : Plus créatif, plus aléatoire

In [ ]:
# Comparer différentes températures
temperatures = [0.3, 0.7, 1.0, 1.5, 2.0]

print("=" * 60)
print("EFFET DE LA TEMPÉRATURE")
print("=" * 60)

for temp in temperatures:
    print(f"\nTempérature = {temp}:")
    for _ in range(5):
        name = generate(model, prompt="", temperature=temp)
        print(f"  {name}")

In [ ]:
# TODO: Répondez aux questions suivantes

# 1. Quelle température donne les noms les plus "réalistes" ?
# Réponse: ...

# 2. Quelle température donne les noms les plus "originaux" ?
# Réponse: ...

# 3. Pourquoi une température trop basse peut poser problème ?
# Réponse: ...

---

# PARTIE 2 : Fine-tuning GPT-2

---

Maintenant, comparons avec un "vrai" modèle de langage pré-entraîné.

## 8. Charger GPT-2

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
# Charger GPT-2 (version petite)
model_name = "gpt2"

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_name)
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

model_gpt2 = GPT2LMHeadModel.from_pretrained(model_name).to(device)

num_params_gpt2 = sum(p.numel() for p in model_gpt2.parameters())
print(f"Modèle: {model_name}")
print(f"Paramètres GPT-2: {num_params_gpt2:,}")
print(f"Paramètres Mini-GPT: {num_params:,}")
print(f"\nRatio: GPT-2 est {num_params_gpt2/num_params:.0f}x plus grand !")

## 9. Génération sans fine-tuning

In [ ]:
from transformers import pipeline

# Créer un pipeline de génération
generator = pipeline(
    'text-generation',
    model=model_gpt2,
    tokenizer=tokenizer_gpt2,
    device=0 if torch.cuda.is_available() else -1
)

# Tester
print("Génération GPT-2 (sans fine-tuning):")
print("-" * 50)

prompts = ["The wizard named", "In the realm of dragons,"]

for prompt in prompts:
    result = generator(prompt, max_length=50, num_return_sequences=1, temperature=0.8)
    print(f"\nPrompt: {prompt}")
    print(f"Généré: {result[0]['generated_text']}")

## 10. Fine-tuning sur notre corpus

In [ ]:
# Préparer le corpus pour GPT-2
# On crée un fichier texte avec un nom par ligne
with open('names_for_gpt2.txt', 'w') as f:
    for name in names:
        f.write(name + '\n')

print(f"Corpus créé: {len(names)} noms")

In [ ]:
# Créer le dataset pour fine-tuning
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

train_dataset = load_dataset('names_for_gpt2.txt', tokenizer_gpt2, block_size=32)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_gpt2,
    mlm=False  # GPT-2 n'utilise pas le MLM
)

print(f"Dataset créé: {len(train_dataset)} exemples")

In [ ]:
# Configuration du fine-tuning
training_args = TrainingArguments(
    output_dir="./gpt2-fantasy",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    warmup_steps=100,
)

trainer = Trainer(
    model=model_gpt2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
# Fine-tuning
print("Fine-tuning GPT-2...")
trainer.train()
print("\nFine-tuning terminé !")

In [ ]:
# Tester après fine-tuning
print("Génération GPT-2 (APRÈS fine-tuning):")
print("-" * 50)

generator_finetuned = pipeline(
    'text-generation',
    model=model_gpt2,
    tokenizer=tokenizer_gpt2,
    device=0 if torch.cuda.is_available() else -1
)

# Générer des noms
for _ in range(10):
    result = generator_finetuned(
        "", 
        max_length=15, 
        num_return_sequences=1, 
        temperature=0.8,
        pad_token_id=tokenizer_gpt2.eos_token_id
    )
    print(result[0]['generated_text'].strip().split('\n')[0])

## 11. Paramètres de génération

### Exercice 5 : Explorer les paramètres

In [ ]:
# TODO: Expérimentez avec différents paramètres
# - temperature: 0.5, 1.0, 1.5
# - top_k: 10, 50, 100
# - top_p: 0.9, 0.95, 1.0
# - repetition_penalty: 1.0, 1.2, 1.5

def generate_with_params(prompt="", temperature=1.0, top_k=50, top_p=0.95, repetition_penalty=1.0):
    result = generator_finetuned(
        prompt,
        max_length=20,
        num_return_sequences=5,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        pad_token_id=tokenizer_gpt2.eos_token_id,
        do_sample=True
    )
    return [r['generated_text'].strip().split('\n')[0] for r in result]

# Test
print("Temperature basse (0.5):")
for name in generate_with_params(temperature=0.5):
    print(f"  {name}")

print("\nTemperature haute (1.5):")
for name in generate_with_params(temperature=1.5):
    print(f"  {name}")

---

## 12. Conclusion

### Comparaison des approches

| Aspect | Mini-GPT (From Scratch) | GPT-2 (Fine-tuning) |
|--------|------------------------|--------------------|
| Paramètres | ~50K | ~124M |
| Temps d'entraînement | ~2 min | ~5 min |
| Qualité | Bonne pour noms simples | Excellente |
| Compréhension | ✅ Totale | ⚠️ Boîte noire |
| Flexibilité | Limité aux noms | Texte libre |

### Ce que vous avez appris

1. **Masque causal** : Comment empêcher le modèle de "tricher" en regardant le futur
2. **Décodeur Transformer** : L'architecture derrière GPT
3. **Génération autoregressive** : Prédire token par token
4. **Température** : Contrôler la créativité
5. **Fine-tuning** : Adapter un modèle pré-entraîné à un domaine spécifique

### Lien avec ChatGPT

ChatGPT utilise la même architecture de base, mais avec :
- **Beaucoup plus de paramètres** (~175B pour GPT-3)
- **RLHF** (Reinforcement Learning from Human Feedback)
- **Instruction tuning** pour suivre les consignes

### Limites

- Les modèles génèrent du texte **statistiquement probable**, pas **vrai**
- Risque d'**hallucinations** (inventer des faits)
- **Biais** présents dans les données d'entraînement

In [ ]:
# Récapitulatif final
print("=" * 60)
print("RÉCAPITULATIF")
print("=" * 60)
print(f"\nMini-GPT (From Scratch):")
print(f"  - Paramètres: {num_params:,}")
print(f"  - Vocabulaire: {vocab_size} caractères")
print(f"  - Dataset: {len(names)} noms fantasy/celtes")

print(f"\nGPT-2 (Fine-tuned):")
print(f"  - Paramètres: {num_params_gpt2:,}")
print(f"  - Ratio: {num_params_gpt2/num_params:.0f}x plus grand")

print("\n" + "=" * 60)
print("Bravo ! Vous avez construit votre propre Mini-GPT !")
print("=" * 60)